# Argus Bird Video Demo
The goal of this demo is to download a movie and extract clips containing birds.

The code below will
- download a short movie of a bird house
- extract sample images
- allow you to mark the images as good(has bird) or bad(no bird)
- find clips of the video that are 'good'
- extract clips


In [ ]:
!pip install git+https://github.com/johnbradley/argus

In [ ]:
import os
import shutil
from IPython.display import display, HTML, Video, Image
from base64 import b64encode
from argus.core import ModelConfig, extract_sample_images, predict_frames, find_clips, extract_clips

## Settings

In [ ]:
# Use the BioCLIP model to create image embeddings
MODEL_CONFIG = ModelConfig(model_name='hf-hub:imageomics/bioclip')
# Alternative model
# MODEL_CONFIG = ModelConfig(model_name='ViT-B-32', pretrained='laion2b_s34b_b79k')

# Use bird movie that is part of https://doi.org/10.6084/m9.figshare.4530839.v1 
BIRD_MOVIE_URL = 'https://figshare.com/ndownloader/files/7336436'
BIRD_MOVIE_PATH = '7336436.mov'
ALL_MOVIE_PATHS = [BIRD_MOVIE_PATH]

# Settings for saving sample images
SAMPLE_DIR = 'birdsamples' # directory to save sample images from the movie
SAMPLE_SECONDS_INTERVAL = 5.0 # seconds to wait between each sample image
MAX_SAMPLES = None # do not limit the number of sample images

# Settings for predicting 'good' or 'bad' movie frames
FRAME_WAIT_SECONDS = 0.5
PREDICTIONS_PATH = "birdframes.csv"
PREDICTIONS_REPORT_PATH = "birdframes.html"
PREDICTIONS_INTERVALS = 2 # record every 2nd prediction for review in the report

# Settings for clips based on frame predictions
CLIPS_PATH = "birdclips.csv"
CLIPS_REPORT_PATH = "birdclips.html"
CLIPS_DIR = "clips"

## Download the bird movie

In [ ]:
if not os.path.exists(BIRD_MOVIE_PATH):
    !wget --output-document=$BIRD_MOVIE_PATH $BIRD_MOVIE_URL
else:
    print("Skipping download since file already exists.")

In [ ]:
def display_video_player(video_path):
    display(Video(url=video_path, width=400))

In [ ]:
display_video_player(BIRD_MOVIE_PATH)

## Extract sample images

In [ ]:
extract_sample_images(sample_dir=SAMPLE_DIR,
                      video_path=BIRD_MOVIE_PATH,
                      seconds_interval = SAMPLE_SECONDS_INTERVAL,
                      max_samples = MAX_SAMPLES)

# STOP
Typically you would stop here, look at some sample images, and manually move images from the 'birdsamples' directory into the 'good' and 'bad' subdirectories. For the demo we will manually copy the files creating a structure as follows.
```
birdsamples/
   good/
     1.png
   bad/
     0.png
     3.png
...
```

Typically you will want more examples than this in the 'good' and 'bad' subdirectories.

In [ ]:
# Move images simulating what a user would do manually
good_filenames = ["1.png"] # has bird
bad_filenames = ["0.png","3.png"] # no bird
print("Good")
for good_filename in good_filenames:
    shutil.copyfile(f"birdsamples/{good_filename}", f"birdsamples/good/{good_filename}")
    display(Image(f"birdsamples/good/{good_filename}", width=400))
print("Bad")    
for bad_filename in bad_filenames:
    shutil.copyfile(f"birdsamples/{bad_filename}", f"birdsamples/bad/{bad_filename}")
    display(Image(f"birdsamples/bad/{bad_filename}", width=400))

## Predict frames based on sample images

In [ ]:
predictions = predict_frames(
    sample_dir=SAMPLE_DIR,
    video_paths=ALL_MOVIE_PATHS,
    model_config=MODEL_CONFIG,
    frame_wait_seconds=FRAME_WAIT_SECONDS,
    report_path=PREDICTIONS_REPORT_PATH,
    report_interval=PREDICTIONS_INTERVALS)
predictions.to_csv(PREDICTIONS_PATH, index=False)
predictions

## Show Predictions report link

In [ ]:
print()
display(HTML(f'<a href="{PREDICTIONS_REPORT_PATH}"><b>Click here to view Predictions Report</b></a>'))
print()

## Merge sequential frames together into clips

In [ ]:
clips_df = find_clips(
    predictions=predictions,
    report_path=CLIPS_REPORT_PATH)
clips_df.to_csv(CLIPS_PATH, index=False)
clips_df

## Show Clips report link

In [ ]:
print()
display(HTML(f'<a href="{CLIPS_REPORT_PATH}"><b>Click here to view Clips Report</b></a>'))
print()

## Extract movies

In [ ]:
clip_paths = extract_clips(clips_df=clips_df, clips_dir=CLIPS_DIR)
clip_paths

In [ ]:
print("\nOriginal video")
display_video_player(BIRD_MOVIE_PATH)
print("\n\n")
for clip_path in clip_paths:
    print("Extracted clip", clip_path)
    display_video_player(clip_path)
    print("\n\n")